In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import Analisa_Harga_Pangan_Seluruh_Indonesia_2018_2019_Part_1_Data_Analysis as DA_I
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import anderson
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon
from scipy.stats import kruskal


Below is statistical test for harga pangan in Indonesia 2018 and 2019

In [2]:
list_daerah = ['aceh', 'bali', 'banten', 'bengkulu', 'diy', 'dki_jakarta', 'gorontalo', 'jabar', 'jambi', 'jateng', 'jatim', 
              'kalbar', 'kalsel', 'kalteng', 'kaltim', 'kalut', 'kepri', 'kepulauan_bangka_belitung', 'lampung', 'maluku',
               'malut', 'ntb', 'ntt', 'papua', 'papua_barat', 'riau', 'sulbar', 'sulsel', 'sulteng', 'sultra', 'sulut', 
               'sumbar', 'sumsel', 'sumut']
#year list
year = ['2018', '2019']

#commodity list
commodity_all = ['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']

In [3]:
#Normality Test
#1. Shapiro-Wilk Test
H0 = 'The data is normally distributed'
commodities = ['Bawang Merah', 'Bawang Putih', 'Beras', 'Cabai Merah',
       'Cabai Rawit', 'Daging Ayam', 'Daging Sapi', 'Gula Pasir',
       'Minyak Goreng', 'Telur Ayam']

#this function will return dictionaries of each province base on inputted data frame year
def shapiro_wilk(data):
    results_0 = {}
    for x in range(len(list_daerah)):
        temp_dict = {}
        for i in range(len(commodities)):
            stat1, pval1 = shapiro(data[list_daerah[x]][commodities[i]])
            temp_dict[commodities[i]] = pval1
        results_0[list_daerah[x]] = temp_dict
    return results_0

#put normality test results inside a dictionary separated by year for each province
result_in_each_province = {}
for i in range(len(year)):
    result_in_each_province[year[i]] = shapiro_wilk(DA_I.all_data_frame_pivot[year[i]])

#examine
#result_in_each_province['2018']['kalut'] 
#result_in_each_province['2018']['kalut']['Daging Sapi']
#good


#this dictionary shows wheter we should use parametric testing or not
alpha = 0.05
choose_test ={}
for a in range(len(list_daerah)):
    temp = {}
    for b in range(len(commodities)):
        if result_in_each_province['2018'][list_daerah[a]][commodities[b]] > 5 and result_in_each_province['2019'][list_daerah[a]][commodities[b]] > 5:
            temp[commodities[b]] = commodities[b]+' should use Parametric testing'
        else:
            temp[commodities[b]] = commodities[b]+' should use Non Parametric testing'
    choose_test[list_daerah[a]] = temp

#examine
choose_test['dki_jakarta']

C:\Users\ADHITYAS\Miniconda3\lib\site-packages\scipy\stats\morestats.py:1673: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


{'Bawang Merah': 'Bawang Merah should use Non Parametric testing',
 'Bawang Putih': 'Bawang Putih should use Non Parametric testing',
 'Beras': 'Beras should use Non Parametric testing',
 'Cabai Merah': 'Cabai Merah should use Non Parametric testing',
 'Cabai Rawit': 'Cabai Rawit should use Non Parametric testing',
 'Daging Ayam': 'Daging Ayam should use Non Parametric testing',
 'Daging Sapi': 'Daging Sapi should use Non Parametric testing',
 'Gula Pasir': 'Gula Pasir should use Non Parametric testing',
 'Minyak Goreng': 'Minyak Goreng should use Non Parametric testing',
 'Telur Ayam': 'Telur Ayam should use Non Parametric testing'}

In [4]:
#Hypothesis testing 
#we will test if there is any significant difference between certain commodity price in the year 2018 and 2019 in the same province
H0 = 'The sample distributions are equal'
all_hypothesis_test_result = {}
for a in range(len(list_daerah)):
    temp_1 = {}
    for b in range(len(commodity_all)):
        #for non parametric testing we use wilcoxon signed rank test
        if 'Non Parametric' in choose_test[list_daerah[a]][commodity_all[b]]:
            #check each data size
            if len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]]) == len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]]):
                stat, p = wilcoxon(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]], 
                                   DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]])
                temp_1[commodity_all[b]] = p
            elif len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]]) < len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]]):
                stat, p = wilcoxon(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]], 
                                   DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]][0: len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]])])
                temp_1[commodity_all[b]] = p
            elif len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]]) > len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]]):
                stat, p = wilcoxon(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]][0: len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]])], 
                                   DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]])
                temp_1[commodity_all[b]] = p
                
        else:
            #for parametric testing we use 2table t-test 
            if len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]]) == len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]]):
                stat, p = ttest_ind(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]], 
                                   DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]])
                temp_1[commodity_all[b]] = p
            elif len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]]) < len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]]):
                stat, p = ttest_ind(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]], 
                                   DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]][0: len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]])])
                temp_1[commodity_all[b]] = p
            elif len(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]]) > len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]]):
                stat, p = ttest_ind(DA_I.all_data_frame_pivot['2018'][list_daerah[a]][commodity_all[b]][0: len(DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]])], 
                                   DA_I.all_data_frame_pivot['2019'][list_daerah[a]][commodity_all[b]])
                temp_1[commodity_all[b]] = p
    all_hypothesis_test_result[list_daerah[a]] = temp_1
    
#examine
#all_hypothesis_test_result['jambi']

all_hypothesis_test_result_transcribed = {}
alpha = 0.05
for a in range(len(list_daerah)):
    temp_1 = {}
    for b in range(len(commodity_all)):
        if all_hypothesis_test_result[list_daerah[a]][commodity_all[b]] > alpha:
            temp_1[commodity_all[b]] = f'{commodity_all[b]} Same distribution (fail to reject H0)'
        else:
            temp_1[commodity_all[b]] = f'{commodity_all[b]} Different distribution (reject H0)'
    all_hypothesis_test_result_transcribed[list_daerah[a]] = temp_1

all_hypothesis_test_result_transcribed['jateng']

C:\Users\ADHITYAS\Miniconda3\lib\site-packages\scipy\stats\morestats.py:2882: RuntimeWarning: invalid value encountered in greater
  r_plus = np.sum((d > 0) * r, axis=0)
C:\Users\ADHITYAS\Miniconda3\lib\site-packages\scipy\stats\morestats.py:2883: RuntimeWarning: invalid value encountered in less
  r_minus = np.sum((d < 0) * r, axis=0)


{'Bawang Merah': 'Bawang Merah Different distribution (reject H0)',
 'Bawang Putih': 'Bawang Putih Different distribution (reject H0)',
 'Beras': 'Beras Different distribution (reject H0)',
 'Cabai Merah': 'Cabai Merah Same distribution (fail to reject H0)',
 'Cabai Rawit': 'Cabai Rawit Same distribution (fail to reject H0)',
 'Daging Ayam': 'Daging Ayam Different distribution (reject H0)',
 'Daging Sapi': 'Daging Sapi Different distribution (reject H0)',
 'Gula Pasir': 'Gula Pasir Same distribution (fail to reject H0)',
 'Minyak Goreng': 'Minyak Goreng Different distribution (reject H0)',
 'Telur Ayam': 'Telur Ayam Different distribution (reject H0)'}

In [5]:
#Hypothesis testing 2
#we will test if there is any significant difference in certain commodity price from every province
#we test 2018 and 2019 separately

#2018
#since we have 34 province that we want to compare for each commodities, 
#we have to have all data to be normally distributed, since most of our data doesnt have gaussian distribution,
#we are going to use kruskal wallis test for non parametric test, it is equivalent to annova in parametric test
list_daerah2 = ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'Daerah Istimewa Yogyakarta', 'DKI Jakarta', 'Gorontalo', 'Jawa Barat', 
                'Jambi', 'Jawa Tengah', 'Jawa Timur', 'Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 
                'Kalimantan Timur', 'Kalimantan Utara', 'Kepulauan Riau', 'Kepulauan Bangka Belitung', 'Lampung', 'Maluku',
               'Maluku Utara', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur', 'Papua', 'Papua Barat', 'Riau', 
                'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara', 
                'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']

H0 = 'The sample distributions are equal'
kruskal_wallis_results = {}
for i in range(len(year)):
    temp_0 = {}
    for a in range(len(commodity_all)):
        stat, p = kruskal(DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[0]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[1]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[2]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[3]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[4]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[5]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[6]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[7]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[8]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[9]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[10]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[11]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[12]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[13]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[14]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[15]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[16]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[17]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[18]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[19]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[20]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[21]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[22]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[23]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[24]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[25]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[26]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[27]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[28]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[29]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[30]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[31]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[32]],
                          DA_I.all_data_frame_pivot_commodity[year[i]][commodity_all[a]][list_daerah2[33]],
                         )
        temp_0[commodity_all[a]] = p
    kruskal_wallis_results[year[i]] = temp_0

#kruskal_wallis_results['2018']['Bawang Putih']
all_kruskal_wallis_test_result_transcribed = {}
alpha = 0.05
for a in range(len(year)):
    temp_1 = {}
    for b in range(len(commodity_all)):
        if kruskal_wallis_results[year[a]][commodity_all[b]] > alpha:
            temp_1[commodity_all[b]] = f'{commodity_all[b]} Same distribution (fail to reject H0)'
        else:
            temp_1[commodity_all[b]] = f'{commodity_all[b]} Different distribution (reject H0)'
    all_kruskal_wallis_test_result_transcribed[year[a]] = temp_1
    
    
all_kruskal_wallis_test_result_transcribed['2018'] 
all_kruskal_wallis_test_result_transcribed['2019'] 

{'Bawang Merah': 'Bawang Merah Different distribution (reject H0)',
 'Bawang Putih': 'Bawang Putih Different distribution (reject H0)',
 'Beras': 'Beras Different distribution (reject H0)',
 'Cabai Merah': 'Cabai Merah Different distribution (reject H0)',
 'Cabai Rawit': 'Cabai Rawit Different distribution (reject H0)',
 'Daging Ayam': 'Daging Ayam Different distribution (reject H0)',
 'Daging Sapi': 'Daging Sapi Different distribution (reject H0)',
 'Gula Pasir': 'Gula Pasir Different distribution (reject H0)',
 'Minyak Goreng': 'Minyak Goreng Different distribution (reject H0)',
 'Telur Ayam': 'Telur Ayam Different distribution (reject H0)'}

i dont have any insight yet, feel free to give me any

and if you want to run this script on your local machines, just clone it

and send me email if you want all CSV data

s.adhitya.dimas@gmail.com